In [ ]:
import os, json
import numpy as np
from PIL import Image
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from pycococreatortools import pycococreatortools
from icevision import models, parsers, show_records, tfms, Dataset, Metric, COCOMetric, COCOMetricType
from icevision.imports import *
from icevision.utils import *
from icevision.data import *
from icevision.metrics.metric import *
from sklearn.metrics import confusion_matrix, f1_score

In [ ]:
model_type = models.torchvision.mask_rcnn
backbone = model_type.backbones.resnet34_fpn()

In [ ]:
data_path = "/root/"
mount_path = "/root/data"

In [ ]:
parser = parsers.COCOMaskParser(annotations_filepath=f"{data_path}/tile-cerulean-v2-partial-with-context/instances_Tiled Cerulean Dataset V2.json", img_dir=f"{mount_path}/tile-cerulean-v2-partial-with-context/tiled_images")


### Important! 

Make sure you have copied the dataset to the local SSD of the VM at /root. Loading the data from a GCP bucket takes a full 2 minutes compared to 17 seconds when data is on the SSD.

In [ ]:
train_records, valid_records = parser.parse(autofix=False)

In [ ]:
class_map = {
    "Infrastructure": 1,
    "Natural Seep": 2,
    "Coincident Vessel": 3,
    "Recent Vessel": 4,
    "Old Vessel": 5,
    "Ambiguous": 6,
    "Hard Negatives": 0,
}

In [ ]:
x=show_records(train_records[0:15], ncols=3, class_map=class_map)

Normalizing is best practice and necessary for icevision to properly display predicition results

In [ ]:
train_tfms = tfms.A.Adapter(
    [
        tfms.A.Normalize(),
    ]
)

In [ ]:
valid_tfms = tfms.A.Adapter([*tfms.A.resize_and_pad(size=512), tfms.A.Normalize()])

sourced from: https://airctic.com/0.8.1/getting_started_instance_segmentation/

In [ ]:
__all__ = ["IoUMetric", "IoUMetricType"]


class IoUMetricType(Enum):
    """Available options for `COCOMetric`."""

    bbox = "bbox"
    mask = "segm"
    keypoint = "keypoints"


class IoUMetric(Metric):
    """Wrapper around [cocoapi evaluator](https://github.com/cocodataset/cocoapi)
    Calculates average precision.
    # Arguments
        metric_type: Dependent on the task you're solving.
        print_summary: If `True`, prints a table with statistics.
        show_pbar: If `True` shows pbar when preparing the data for evaluation.
    """

    def __init__(
        self,
        metric_type: IoUMetricType = IoUMetricType.mask,
        iou_thresholds: Optional[Sequence[float]] = None,
        print_summary: bool = False, 
        show_pbar: bool = True,
    ):
        self.metric_type = metric_type
        self.iou_thresholds = iou_thresholds
        self.print_summary = print_summary
        self.show_pbar = show_pbar
        self._records, self._preds = [], []

    def _reset(self):
        self._records.clear()
        self._preds.clear()

    def accumulate(self, preds):
        for pred in preds:
            self._records.append(pred.ground_truth)
            self._preds.append(pred.pred)

    def finalize(self) -> Dict[str, float]:
        with CaptureStdout():
            coco_eval = create_coco_eval(
                records=self._records,
                preds=self._preds,
                metric_type=self.metric_type.value,
                iou_thresholds=self.iou_thresholds,
                show_pbar=self.show_pbar,
            )
            coco_eval.evaluate()
            coco_eval.accumulate()

        with CaptureStdout(propagate_stdout=self.print_summary):
            coco_eval.summarize()

        stats = coco_eval.stats
        ious = coco_eval.ious

        ious_l = []
        for iou in ious.values():
            if isinstance(iou, np.ndarray):
                iou = iou.tolist()
            else:
                iou = iou
            ious_l.append(iou)
        
        flat_ious_l = [item for sublist in ious_l for item in sublist]
        if len(flat_ious_l) == 0:
            flat_ious_l.append([0])
        flat_ious_l = [item for items in flat_ious_l for item in items]
        ious_avg = np.array(flat_ious_l).mean()
        ious_min = np.array(flat_ious_l).min()
        ious_max = np.array(flat_ious_l).max()
        
        logs = {
            #"Min IoU area=all": ious_min,
            #"Max IoU area=all": ious_max,
            "Avg. IoU area=all": ious_avg, 
        }
        self._reset()
        
        
        return logs

In [ ]:
train_ds = Dataset(train_records, train_tfms)
valid_ds = Dataset(valid_records, valid_tfms)

train_dl = model_type.train_dl(train_ds, batch_size=8, num_workers=6, shuffle=True) # adjust num_workers for your processor count
valid_dl = model_type.valid_dl(valid_ds, batch_size=8, num_workers=6, shuffle=False)

infer_dl = model_type.infer_dl(valid_ds, batch_size=8, shuffle=False)

model = model_type.model(backbone=backbone(pretrained=False), num_classes=len(parser.class_map))

metrics = [IoUMetric(metric_type=IoUMetricType.mask)]

learn = model_type.fastai.learner(dls=[train_dl, valid_dl], model=model, metrics=metrics)

In [ ]:
lr = learn.lr_find()

In [ ]:
lr.valley

LR finder takes about a minute when data is on SSD

1 train epoch is about 4 minutes. 1 validation epoch of 76 samples is also about a minute.

In [ ]:
#learn.fine_tune(10, 3e-3) # 3e-3 is hand selected lr
learn.fine_tune(10, lr.valley) #, freeze_epochs=2)

a TODO is to debug the COCOMetric, it should not be -1 given that we are now acheiving detections that intersect with groundtruth.

In [ ]:
!mkdir {mount_path}/experiments/cv2/05062022_ep10

In [ ]:
torch.save(model, f'{mount_path}/experiments/cv2/05062022_ep10/05062022_ep10.pth')

In [ ]:
print(f"approximate time to train 30 epochs in minutes: {25*30/60}")


The predictions above .7 confidence that roughly line up with groundtruth demonstrates that icevision-trained models can produce predictions that look like they are headed in the correct direction, even for an imperfect training set.

In [ ]:
model_type.show_results??

In [ ]:
model.eval()

In [ ]:
x = model_type.show_results(model, valid_ds, detection_threshold=.6) 
plt.savefig("inference_results.png")

In [ ]:
preds = model_type.predict_from_dl(model, infer_dl, keep_images=True)

In [ ]:
d = preds[10].as_dict()

In [ ]:
d

In [ ]:
v = valid_records[10].as_dict()

In [ ]:
v

In [ ]:
v_masks = v['detection']['masks'] #[0].to_mask(d['common']['height'],d['common']['width']).data

In [ ]:
d_masks = d['detection']['masks'] #[0].to_mask(d['common']['height'],d['common']['width']).data

In [ ]:
# flatten our mask arrays and use scikit-learn to create a confusion matrix
flat_preds = np.concatenate(d_masks).flatten()
flat_truth = np.concatenate(v_masks).flatten()
OUTPUT_CHANNELS = 6
cm = confusion_matrix(flat_truth, flat_preds, labels=list(range(OUTPUT_CHANNELS)))

classes = [1,2,3,4,5,6]

#%matplotlib inline
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
fig, ax = plt.subplots(figsize=(10, 10))
im = ax.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
ax.figure.colorbar(im, ax=ax)
# We want to show all ticks...
ax.set(xticks=np.arange(cm.shape[1]),
       yticks=np.arange(cm.shape[0]),
       # ... and label them with the respective list entries
       xticklabels=list(range(OUTPUT_CHANNELS)), yticklabels=list(range(OUTPUT_CHANNELS)),
       title='Normalized Confusion Matrix',
       ylabel='True label',
       xlabel='Predicted label')

# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

# Loop over data dimensions and create text annotations.
fmt = '.2f' #'d' # if normalize else 'd'
thresh = cm.max() / 2.
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        ax.text(j, i, format(cm[i, j], fmt),
                ha="center", va="center",
                color="white" if cm[i, j] > thresh else "black")
fig.tight_layout(pad=2.0, h_pad=2.0, w_pad=2.0)
ax.set_ylim(len(classes)-0.5, -0.5)

plt.savefig(f'{ROOT_DIR}cm.png')


# compute f1 score
f1 = f1_score(flat_truth, flat_preds, average='macro')

print("cm: ", cm)
print("f1: ", f1)
